# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
### Lab 1. Object detection 데이터셋 준비하기

#### 랩 순서

1. 샘플 이미지들 S3에서 받아서, 여러분의 S3에 올리기
2. Amazon SageMaker Ground Truth 레이블링 작업 생성하기
3. 레이블링 툴을 사용해서 레이블링하기
4. GluonCV 학습 데이터셋으로 변환하기

In [17]:
#S3_BUCKET= 'mh-s3-icn'
S3_BUCKET= 'jk-s3-iad'
S3_PREFIX = 'ground-truth-od-full-demo-lab'
#GT_JOB_NAME = 'devday-gluoncv-lab'
GT_JOB_NAME = 'devday-gluoncv-lab-2'

### Task 1 - 데이터셋 업로드하기

In [6]:
#!wget https://mh-s3-icn.s3.ap-northeast-2.amazonaws.com/devday/aws-devday-gluoncv-dataset-lab.tar.gz
#!wget https://jk-s3-iad.s3.amazonaws.com/devday/aws-devday-gluoncv-dataset-lab.tar.gz
!rm -rf ./lab-images; mkdir ./lab-images
!tar -C ./lab-images -xzf ./aws-devday-gluoncv-dataset-lab.tar.gz

- <span style="color:red">(09/10, JK) tar.gz 파일 다운로드 링크 수정 필요. wget으로 download 안하고 git clone하면서 저절로 따라오게?</span>
- <span style="color:red">(09/10, JK) image 파일들을 s3로 업로드하는 부분 설명이 없음.</span>

### Task 2 - Amazon SageMaker Ground Truth 레이블링 작업 만들기

#### 1. "Create labelling job" 버튼 클릭

![alt text](./images/devday-gt-lab-01.png)

---
#### 2. Job 내용 입력하기

- **Job name**: 레이블링 job 이름
- **Image dataset location**: 레이블링 할 이미지들이 있는 S3 위치 지정하기

- <span style="color:red">(09/10, JK) 아래 3번으로 넘어가려면 "Create manifest file"을 클릭해야 하는데 아래 스크린샷에서 그 부분을 강조해주면 좋을 듯함.</span>
- <span style="color:red">(09/10, JK) console language를 한글로 설정하는 고객이 매우 많으므로 영어로 전환하는 방법의 공지가 필요함.</span>

![alt text](./images/devday-gt-lab-02.png)

---
#### 3. **Manifest file** 만들기

- **Input dataset location**: s3://<bucket 이름>/<image 파일이 있는 prefix>/
- **Data type**: Images 선택
- "Create" 클릭

![](./images/devday-gt-lab-03.png)

- <span style="color:red">(09/10, JK) 위의 경로가 미리 만들어져 있지 않으면 (= s3에 upload가 안되면) error 메시지가 남.  </span>

* "Use this manifest" 선택하기

![](./images/devday-gt-lab-04.png)

위 단계들을 마친 후, 

- **Output dataset location**: 레이블링 결과를 저장할 S3 위치 지정하기
- **IAM Role**: 레이블링 job이 이용할 IAM Role 지정하기

를 채웁니다.

- <span style="color:red">(09/10, JK) Output dataset location에서 수정할 부분과 그냥 둘 부분(/outputs)를 구분해줘야할 듯함.  </span>
- <span style="color:red">(09/10, JK) IAM Role은 "Create a new role" 을 선택하면 된다고 알려줘야 함. Create an IAM role 대화 상자의 스크린 샷도 필요할 듯. </span>

![](./images/devday-gt-lab-05.png)

---
#### 4. 태스크 종류 고르기

Object detection 모델을 위한 데이터셋 레이블링 작업 생성이기 때문에, "Bounding box"를 선택하고 아래 Next 버튼을 클릭합니다.

![](./images/devday-gt-lab-06.png)

---
#### 5. 작업자(worker) 선택하기

* **Worker types**: 직접 레이블링을 수행하기 위해서 "Private"을 선택합니다.
* **Team name**: 레이블링 작업자들에 대한 팀 이름을 지정합니다/
* **Invite private annotators**: 레이블링을 수행할 사람들의 이메일 주소들을 입력합니다. 여러 이메일 주소를 입력할 경우 ","로 구분합니다.

이 후에 작업자를 더 추가하고 싶으면 [SageMaker > Ground Truth > Labelling workforces]를 선택한 후, 'Private' > 'Workers' > 'Invite new workers'를 통해서 추가할 수 있습니다.


- <span style="color:red">(09/10, JK) 아래 스크린샷에서 이메일 주소 masking 필요함 (2개).</span>

![](./images/devday-gt-lab-07.png)

![](./images/devday-gt-lab-08.png)

- <span style="color:red">(09/10, JK) 위 화면에서 Submit을 누르면 "Labeling job devday-gluoncv-lab was successfully created." 라는 콘솔로 화면이 바뀌는데 스크린샷 필요.</span>

---
#### 6. 작업자 등록 이메일 확인하기

작업자로 등록한 이메일로 로그인 정보(사용자 이름, 임시 비밀번호)와 레이블링 툴 URL이 전달됩니다. 해당 URL을 통해서 레이블링 작업을 시작할 수 있습니다.

레이블링 툴 URL은 [SageMaker > Ground Truth > Labelling workforces]를 선택한 후, 'Private' 탭의 'Private workforce summary'의 'Labeling portal sign-in URL'에서도 확인할 수 있습니다.

![](./images/devday-gt-lab-09.png)

---
#### 7. 레이블링 툴을 이용해서 레이블링 하기

툴 URL로 로그인하면, 본인에게 할당된 작업 목록이 아래와 같이 나옵니다. 하나를 선택한 후, "Start working" 버튼을 누르면 레이블링을 시작합니다.

![](./images/devday-gt-lab-10.png)

Object detection 모델을 위한 레이블링 작업으로, 그림에서 오른쪽에 나열된 객체를 선택해서 네모 상자(bounding box)로 레이블을 지정한 후 "Submit" 버튼을 눌려서 다음 이미지로 넘어갑니다. 만약 해당 레이블이 없을 경우에는 "Nothing to label"을 선택하고 "Submit"을 클릭합니다.

![](./images/devday-gt-lab-11.png)

---
#### 8. 레이블링 결과 확인하기

[SageMaker > Ground Truth > Labelling jobs]에서 해당 레이블링 잡을 선택하면 아래와 같이 레이블링 결과를 확인 할 수 있습니다. 

![](./images/devday-gt-lab-12.png)

"Output dataset location"으로 지정된 S3에는 아래와 같은 결과들이 저장되어 있습니다.

* annotation-tool
* annotations: 작업자들이 수행한 모든 레이블링 결과를 저장하고 있음
* manifests: 레이블링 잡의 output manifest들이 저장되어 있음. 최종 결과에 대한 manifest는 **outputs/output.manifest**로 저정되어 있음


**manifests/outputs/output.manifest** 각 이미지들에 대한 레이블링 결과를 JSON 형태로 저장하고 있습니다.

![](./images/devday-gt-lab-14.png)

```json
{
	"source-ref":"s3://mh-s3-icn/ground-truth-od-full-demo-lab/images/000132c20b84269b.jpg",
	"devday-gluoncv-lab":
	{
		"annotations":[
			{"class_id":3,"width":205,"top":182,"height":583,"left":689},
			{"class_id":3,"width":158,"top":125,"height":643,"left":564},
			{"class_id":3,"width":268,"top":197,"height":571,"left":21},
			{"class_id":3,"width":267,"top":124,"height":641,"left":328}],
		"image_size":[{"width":1024,"depth":3,"height":768}]
	},
	"devday-gluoncv-lab-metadata":
	{
		"job-name":"labeling-job/devday-gluoncv-lab",
		"class-map":{"1":"Boat","0":"Bird","3":"Person","2":"Car"},
		"human-annotated":"yes","objects":[
			{"confidence":0.09},
			{"confidence":0.09},
			{"confidence":0.09},
			{"confidence":0.09}],
		"creation-date":"2019-09-08T21:38:34.068123",
		"type":"groundtruth/object-detection"
	}
}

```

---
#### 9. GluonCV 데이터셋으로 변환하기


In [18]:
!aws s3 cp s3://$S3_BUCKET/$S3_PREFIX/outputs/$GT_JOB_NAME/manifests/output/output.manifest .

download: s3://jk-s3-iad/ground-truth-od-full-demo-lab/outputs/devday-gluoncv-lab-2/manifests/output/output.manifest to ./output.manifest


In [16]:
!wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py

--2019-09-10 10:22:53--  https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15838 (15K) [text/plain]
Saving to: ‘im2rec.py.1’

im2rec.py.1         100%[===================>]  15.47K  --.-KB/s    in 0s      

2019-09-10 10:22:53 (45.5 MB/s) - ‘im2rec.py.1’ saved [15838/15838]

